# 平均報酬強化学習

参考：
* [Model-free Reinforcement Learning in Infinite-horizon Average-reward Markov Decision Processes](https://arxiv.org/abs/1910.07072)

今回は平均報酬強化学習におけるモデルフリーな探索を見てみます．今回はWeakly communicatingを考えます．

Weakly communicatingの条件はほぼ必須です．（[REGAL: A Regularization based Algorithm for Reinforcement Learning in Weakly Communicating MDPs](https://arxiv.org/abs/1205.2661)を読もう）

基本的には，平均報酬強化学習を割引報酬強化学習で近似することでリグレットを導出していきます．

表記：
* 平均総報酬：$J^\pi(s):=\liminf _{T \rightarrow \infty} \frac{1}{T} \mathbb{E}\left[\sum_{t=1}^T r\left(s_t, \pi\left(s_t\right)\right) \mid s_1=s\right]$
    * 最適ベルマン方程式：$J^*+q^*(s, a)=r(s, a)+\mathbb{E}_{s^{\prime} \sim p(\cdot \mid s, a)}\left[v^*\left(s^{\prime}\right)\right]$
    * Weakly communicatingでは全ての$s$で$J^*(s)=J^*$な$J^*$が存在する．
    * ここで，$q^*$は（定数項を除けば）唯一に定まる．
* リグレット：$R_T:=\sum_{t=1}^T\left(J^*-r\left(s_t, a_t\right)\right)$．最適な平均報酬から，道中で出会った報酬を引いた値．
* Span：$\operatorname{sp}\left(v^*\right)=\max _s v^*(s)-\min _s v^*(s)$
* 割引報酬の表記：
$$
\begin{aligned}
\forall(s, a), & Q^*(s, a) & =r(s, a)+\gamma \mathbb{E}_{s^{\prime} \sim p(\cdot \mid s, a)}\left[V^*\left(s^{\prime}\right)\right] \\
\forall s, & V^*(s) & =\max _{a \in \mathcal{A}} Q^*(s, a) .
\end{aligned}
$$

---

仮定：

**Weakly communicating MDP**

MDPの状態が次の２つに分割できる場合，そのMDPのことをWeakly communicatingと呼びます．
1. すべての状態が，任意の定常方策について，transientである．
2. 任意の２つの状態を行き来できる定常方策が存在する．

この仮定がない場合は平均報酬でLow regretを達成するのは無理らしいです．

---

## 割引アルゴリズム

元論文の証明を簡単にするため，次のアルゴリズムを考えます．

1. パラメータ：$\gamma \in [0, 1)$, $\delta \in (0, 1)$
2. 初期化：
    * $H = 1 / (1-\gamma), \quad \forall s: \hat{V}_1(s)=H$
    * $\forall s, a: Q_1(s, a)=\hat{Q}_1(s, a)=H, \quad n_1(s, a)=0$
3. $\forall \tau, \alpha_\tau=\frac{H+1}{H+\tau}, b_\tau=2 \operatorname{sp}\left(V^*\right) \sqrt{\frac{H}{\tau} \ln \frac{2 T}{\delta}}$
4. For t = 1...T, do
    * $a_t=\operatorname{argmax}_{a \in \mathcal{A}} \hat{Q}_t\left(s_t, a\right)$
    * 次の更新をします：

$$
\begin{aligned}
& n_{t+1}\left(s_t, a_t\right) \leftarrow n_t\left(s_t, a_t\right)+1 \\
& \tau \leftarrow n_{t+1}\left(s_t, a_t\right) \\
& Q_{t+1}\left(s_t, a_t\right) \leftarrow\left(1-\alpha_\tau\right) Q_t\left(s_t, a_t\right) 
+\alpha_\tau\left[r\left(s_t, a_t\right)+\gamma \hat{V}_t\left(s_{t+1}\right)+b_\tau\right]\\
& \hat{Q}_{t+1}\left(s_t, a_t\right) \leftarrow \min \left\{\hat{Q}_t\left(s_t, a_t\right), Q_{t+1}\left(s_t, a_t\right)\right\} \\
& \hat{V}_{t+1}\left(s_t\right) \leftarrow \max _{a \in \mathcal{A}} \hat{Q}_{t+1}\left(s_t, a\right) .
\end{aligned}
$$

ここで，
* $\hat{Q}_t$は$Q^*$の推定用
* $\hat{V}_t$は$V^*$の推定用
* $b_\tau$はボーナス

です．

### リグレットの導出

次のバウンドを出します．つまり，割引MDPでの平均報酬のバウンドです．

$$
\begin{aligned}
& \sum_{t=1}^T(1-\gamma)V^*\left(s_t\right)-r\left(s_t, a_t\right) \\
=& \sum_{t=1}^T\left(V^*\left(s_t\right)-Q^*\left(s_t, a_t\right)\right) \\
& +\sum_{t=1}^T\left(Q^*\left(s_t, a_t\right)-\gamma V^*\left(s_t\right)-r\left(s_t, a_t\right)\right) .
\end{aligned}
$$

### １つ目のバウンド

まず，$\hat{Q}_{t}$と$Q^*$の差をバウンドします．

確率$1-\delta$以上で，任意の$t=1,\dots, T$と$(s, a)$について，

$$
0 \leq \hat{Q}_{t+1}(s, a)-Q^*(s, a) \leq H \alpha_\tau^0+\gamma \sum_{i=1}^\tau \alpha_\tau^i\left[\hat{V}_{t_i}\left(s_{t_i+1}\right)-V^*\left(s_{t_i+1}\right)\right]+6 \operatorname{sp}\left(V^*\right) \sqrt{\frac{H}{\tau} \ln \frac{2 T}{\delta}}
$$

が成立します．ここで，$\tau=n_{t+1}(s, a)$は$(s, a)$を最初の$t$ ステップで訪れる総回数を表します．また，
$$\alpha_\tau^0:=\prod_{j=1}^\tau\left(1-\alpha_j\right), \quad \alpha_\tau^i:=\alpha_i \prod_{j=i+1}^\tau\left(1-\alpha_j\right)$$
としました．$t_1, \dots, t_\tau \leq t$は$(s, a)$が訪問された時刻です．

**証明**

アルゴリズムの定義から，

$$
Q_{t+1}(s, a)=H \alpha_\tau^0+\sum_{i=1}^\tau \alpha_\tau^i\left[r(s, a)+\gamma \hat{V}_{t_i}\left(s_{t_i+1}\right)\right]+\sum_{i=1}^\tau \alpha_\tau^i b_i
$$

です．
ここで，$\sum_{i=0}^\tau \alpha_\tau^i=1$が成り立つので（論文だと$i=1$になってるけど，多分$i=0$だと思う），
$$
Q^*(s, a)=\alpha_\tau^0 Q^*(s, a)+\sum_{i=1}^\tau \alpha_\tau^i\left[r(s, a)+\gamma \mathbb{E}_{s^{\prime} \sim p(\cdot \mid s, a)} V^*\left(s^{\prime}\right)\right]
$$
が成立します．

以上の２つの式の差を取って，
$$
\begin{aligned}
&Q_{t+1}(s, a)-Q^*(s, a)\\
&=\alpha_\tau^0\left(H-Q^*(s, a)\right) \\
&+\gamma \sum_{i=1}^\tau \alpha_\tau^i\left[\hat{V}_{t_i}\left(s_{t_i+1}\right)-V^*\left(s_{t_i+1}\right)\right] \\
&+\gamma \sum_{i=1}^\tau \alpha_\tau^i\left[V^*\left(s_{t_i+1}\right)-\mathbb{E}_{s^{\prime} \sim p(\cdot \mid s, a)} V^*\left(s^{\prime}\right)\right]+\sum_{i=1}^\tau \alpha_\tau^i b_i .
\end{aligned}
$$

* １項目は$\alpha_\tau^0 H$でバウンドされます
* ３項目はマルチンゲール差分列であり，またその確率変数は$\left[-\gamma \alpha_\tau^i \operatorname{sp}\left(V^*\right), \gamma \alpha_\tau^i \operatorname{sp}\left(V^*\right)\right]$でバウンドされます．（別に$1/(1-\gamma)$でバウンドしてもいいかもだけど，こっちのほうが良い．）
    * よって，Azuma Hoeffdingより，３項目は次でバウンドされます．
    * ここで，状態と行動は$(s_1, a_1) \sim (s_T, a_T)$の$T$回しか変化しないので，$T$についてUnion boundを取れば，$SA$についてもUnion boundを取ることになります．
$$
\gamma \operatorname{sp}\left(V^*\right) \sqrt{2 \sum_{i=1}^\tau\left(\alpha_\tau^i\right)^2 \ln \frac{2 T}{\delta}} \leq
2 \gamma \operatorname{sp}\left(V^*\right) \sqrt{\frac{H}{\tau} \ln \frac{2 T}{\delta}} \leq 4 \gamma \operatorname{sp}\left(v^*\right) \sqrt{\frac{H}{\tau} \ln \frac{2 T}{\delta}}
$$
* ４項目はボーナスの定義を代入すればできます．

最後に，
$$
\hat{Q}_{t+1}(s, a)=\min \left\{\hat{Q}_t(s, a), Q_{t+1}(s, a)\right\} \leq Q_{t+1}(s, a)
$$
を使って全部の上界を合体させれば，上界についての証明は完了です．
下界は省略します．

---


上の補題を使うと，次の事実が示せます：
確率$1-\delta$以上で，

$$
\sum_{t=1}^T\left(V^*\left(s_t\right)-Q^*\left(s_t, a_t\right)\right) \leq 4 H S A+24 \operatorname{sp}\left(v^*\right) \sqrt{H S A T \ln \frac{2 T}{\delta}}
$$

つまり，最適価値のアドバンテージが上からバウンドされます．

**証明**

上で示した補題が成り立ってるとします．
* $n_t \geq 1$を$n_{t+1}(s_t, a_t)$とします．つまり，$(s_t, a_t)$を最初の$t$ラウンドで踏んだ回数です．
* $t_i(s, a)$を$(s, a)$を$i$回目に踏んだ時刻とします．なので$t_\tau(s_t, a_t) = t$ですね．

すると，$a_t=\operatorname{argmax}_a \hat{Q}_t\left(s_t, a\right)$なので，

$$
\begin{aligned}
& \sum_{t=1}^T\left(\hat{V}_t\left(s_t\right)-V^*\left(s_t\right)\right)+\sum_{t=1}^T\left(V^*\left(s_t\right)-Q^*\left(s_t, a_t\right)\right) \\
& =\sum_{t=1}^T\left(\hat{Q}_t\left(s_t, a_t\right)-Q^*\left(s_t, a_t\right)\right) \\
& =\sum_{t=1}^T\left(\hat{Q}_{t+1}\left(s_t, a_t\right)-Q^*\left(s_t, a_t\right)\right)+\sum_{t=1}^T\left(\hat{Q}_t\left(s_t, a_t\right)-\hat{Q}_{t+1}\left(s_t, a_t\right)\right) \\
& \leq \underbrace{\sum_{t=1}^T \alpha_{n_t}^0 H}_{\leq SAH} + 6 \operatorname{sp}\left(V^*\right) \sum_{t=1}^T \sqrt{\frac{H}{n_t} \ln \frac{2 T}{\delta}}+\gamma \sum_{t=1}^T \sum_{i=1}^{n_t} \alpha_{n_t}^i\left[\hat{V}_{t_i\left(s_t, a_t\right)}\left(s_{t_i\left(s_t, a_t\right)+1}\right)-V^*\left(s_{t_i\left(s_t, a_t\right)+1}\right)\right]+\sum_{t=1}^T\left(\hat{Q}_t\left(s_t, a_t\right)-\hat{Q}_{t+1}\left(s_t, a_t\right)\right)
\end{aligned}
$$

が成立します．最後の不等式の１個目では上の補題を使いました．
ここで，２項目をバウンドしましょう．
$$
\begin{aligned}
&\sum^T_{t=1}\hat{Q}_{t}\left(s_t, a_t\right) - \hat{Q}_{t+1}\left(s_t, a_t\right) \\
=& \sum^T_{t=1} \sum_{(s, a) \in \mathcal{S}\times \mathcal{A}} \mathbb{I}[(s_t, a_t)=(s, a)]\left(\hat{Q}_{t}\left(s, a\right) - \hat{Q}_{t+1}\left(s, a\right)\right) \\
=& \sum_{(s, a) \in \mathcal{S}\times \mathcal{A}} \sum^T_{t=1} \mathbb{I}[(s_t, a_t)=(s, a)]\left(\hat{Q}_{t}\left(s, a\right) - \hat{Q}_{t+1}\left(s, a\right)\right) \\
=& \sum_{(s, a) \in \mathcal{S}\times \mathcal{A}} \mathbb{I}[(s_1, a_1)=(s, a)]\hat{Q}_{1}\left(s, a\right) - 
\mathbb{I}[(s_T, a_T)=(s, a)]\hat{Q}_{T+1}\left(s, a\right)\\
\leq & SAH
\end{aligned}
$$

<!-- $$
\hat{Q}_{t+1}\left(s_t, a_t\right) = \min \left\{\hat{Q}_t\left(s_t, a_t\right), \left(1-\alpha_\tau\right) Q_t\left(s_t, a_t\right) 
+\alpha_\tau\left[r\left(s_t, a_t\right)+\gamma \hat{V}_t\left(s_{t+1}\right)+b_\tau\right]\right\} 
$$
なので， -->

<!-- 
ここで，$\tau^{-} = n_{t+1}(s_t, a_t)-1$とおくと，（$s_t, a_t$は時刻$t$で絶対に踏むので）

$$
\begin{aligned}
% Q_{t+1}(s_t, a_t)&=H \alpha_\tau^0+\sum_{i=1}^\tau \alpha_\tau^i\left[r(s_t, a_t)+\gamma \hat{V}_{t_i}\left(s_{t_i+1}\right)\right]+\sum_{i=1}^\tau \alpha_\tau^i b_i\\
Q_{t}(s_t, a_t)&=H \alpha_{\tau^-}^0+\sum_{i=1}^{\tau^-} \alpha_{\tau^-}^i\left[r(s_t, a_t)+\gamma \hat{V}_{t_i}\left(s_{t_i+1}\right)\right]+\sum_{i=1}^{\tau^-} \alpha_{\tau^-}^i b_i
\end{aligned}
$$

であることを踏まえると，
$$
\begin{aligned}
&Q_t\left(s_t, a_t\right)-\left[r\left(s_t, a_t\right)+\gamma \hat{V}_t\left(s_{t+1}\right)+b_\tau\right]\\
=& 
H \alpha_{\tau^-}^0+\sum_{i=1}^{\tau^-} \alpha_{\tau^-}^i\left[r(s_t, a_t)+\gamma \hat{V}_{t_i}\left(s_{t_i+1}\right)\right]+\sum_{i=1}^{\tau^-} \alpha_{\tau^-}^i b_i
- \left(r\left(s_t, a_t\right)+\gamma \hat{V}_t\left(s_{t+1}\right)+b_\tau\right)
\end{aligned}
$$
です．それぞれについて和を取ってみましょう．紛らわしいので，一時的に$\tau_t$って書くことにします．

1. $\sum^T_{t=1}H \alpha_{\tau_t^-}^0 \leq HSA$
2. $\sum^T_{t=1}\sum_{i=1}^{\tau_t^-} \alpha_{\tau_t^-}^i b_i - b_{\tau_t}$：これは多分ボーナスが$1 / \sqrt{\tau}$であることを利用して，Lemma 10でちゃんとバウンドすれば，$\sum^T_{t=1}1/\sqrt{\tau_t}$がでてくるはず．
3. $\sum^T_{t=1} \sum_{i=1}^{\tau^{-}_t} \hat{V}_{t_i}(s_{t+1}) - \hat{V}_t(s_{t+1})$

TODO: **ここちゃんと書いてみよう．元論文はちゃんとしてない．多分$\sum^T_{t=1}$を$\sum_{s, a}$について入れ替えて，固定された$(s, a)$で無限回更新したときの差分の和をバウンドすれば良さそう．** -->



さて，３項目は
$$
\begin{aligned}
& \gamma \sum_{t=1}^T \sum_{i=1}^{n_t} \alpha_{n_t}^i\left[\hat{V}_{t_i\left(s_t, a_t\right)}\left(s_{t_i\left(s_t, a_t\right)+1}\right)-V^*\left(s_{t_i\left(s_t, a_t\right)+1}\right)\right] \\
& =\gamma \sum_{t=1}^T \sum_{s, a} \mathbb{1}_{\left[s_t=s, a_t=a\right]} \sum_{i=1}^{n_{t+1}(s, a)} \alpha_{n_{t+1}(s, a)}^i\left[\hat{V}_{t_i(s, a)}\left(s_{t_i(s, a)+1}\right)-V^*\left(s_{t_i(s, a)+1}\right)\right] \\
& =\gamma \sum_{s, a} \sum_{j=1}^{n_{T+1}(s, a)} \sum_{i=1}^j \alpha_j^i\left[\hat{V}_{t_i(s, a)}\left(s_{t_i(s, a)+1}\right)-V^*\left(s_{t_i(s, a)+1}\right)\right]
\end{aligned}
$$
で変形します．
さらに，

$$
\begin{aligned}
& \gamma \sum_{s, a} \sum_{i=1}^{n_{T+1}(s, a)} \sum_{j=i}^{n_{T+1}(s, a)} \alpha_j^i\left[\hat{V}_{t_i(s, a)}\left(s_{t_i(s, a)+1}\right)-V^*\left(s_{t_i(s, a)+1}\right)\right] \\
& =\gamma \sum_{s, a} \sum_{i=1}^{n_{T+1}(s, a)}\left[\hat{V}_{t_i(s, a)}\left(s_{t_i(s, a)+1}\right)-V^*\left(s_{t_i(s, a)+1}\right)\right] \sum_{j=i}^{n_{T+1}(s, a)} \alpha_j^i
\end{aligned}
$$

を使って，

$$
\begin{aligned}
& \gamma \sum_{s, a} \sum_{i=1}^{n_{T+1}(s, a)}\left[\hat{V}_{t_i(s, a)}\left(s_{t_i(s, a)+1}\right)-V^*\left(s_{t_i(s, a)+1}\right)\right] \sum_{j=i}^{n_{T+1}(s, a)} \alpha_j^i \\
& \leq \gamma \sum_{s, a} \sum_{i=1}^{n_{T+1}(s, a)}\left[\hat{V}_{t_i(s, a)}\left(s_{t_i(s, a)+1}\right)-V^*\left(s_{t_i(s, a)+1}\right)\right] \sum_{j=i}^{\infty} \alpha_j^i \\
& =\gamma \sum_{s, a} \sum_{i=1}^{n_{T+1}(s, a)}\left[\hat{V}_{t_i(s, a)}\left(s_{t_i(s, a)+1}\right)-V^*\left(s_{t_i(s, a)+1}\right)\right]\left(1+\frac{1}{H}\right) \\
& =\left(1+\frac{1}{H}\right) \gamma \sum_{t=1}^T\left[\hat{V}_t\left(s_{t+1}\right)-V^*\left(s_{t+1}\right)\right] \\
& =\left(1+\frac{1}{H}\right) \gamma \sum_{t=1}^T\left[\hat{V}_{t+1}\left(s_{t+1}\right)-V^*\left(s_{t+1}\right)\right]+\left(1+\frac{1}{H}\right) \sum_{t=1}^T\left[\hat{V}_t\left(s_{t+1}\right)-\hat{V}_{t+1}\left(s_{t+1}\right)\right] \\
& \leq \sum_{t=2}^{T+1}\left[\hat{V}_t\left(s_t\right)-V^*\left(s_t\right)\right]+\left(1+\frac{1}{H}\right) S H
\end{aligned}
$$

を出します．最後の不等式の$SH$は前セルの$SAH$みたいなのりかも？
最後に，telescopingを使って，
$$
\begin{aligned}
\sum_{t=1}^T\left(V^*\left(s_t\right)-Q^*\left(s_t, a_t\right)\right) & \leq 12 \operatorname{sp}\left(v^*\right) \sum_{t=1}^T \sqrt{\frac{H}{n_t} \ln \frac{2 T}{\delta}}+\left(\hat{V}_{T+1}\left(s_{T+1}\right)-V^*\left(s_{T+1}\right)\right)+\left(1+\frac{1}{H}\right) S H+S A H \\
& \leq 12 \operatorname{sp}\left(v^*\right) \sum_{t=1}^T \sqrt{\frac{H}{n_t} \ln \frac{2 T}{\delta}}+4 S A H
\end{aligned}
$$
が成り立ちます．後はPotential lemmaより

$$
\sum_{t=1}^T \frac{1}{\sqrt{n_{t+1}\left(s_t, a_t\right)}}=\sum_{t=1}^T \sum_{s, a} \frac{\mathbb{1}_{\left[s_t=s, a_t=a\right]}}{\sqrt{n_{t+1}(s, a)}}=\sum_{s, a} \sum_{j=1}^{n_{T+1}(s, a)} \frac{1}{\sqrt{j}} \leq \sum_{s, a} 2 \sqrt{n_{T+1}(s, a)} \leq 2 \sqrt{S A \sum_{s, a} n_{T+1}(s, a)}=2 \sqrt{S A T}
$$

を使って証明終わりです．


---

**３つ目のバウンド**

確率$1-\delta$以上で次が成立します．

$$
\begin{aligned}
& \sum_{t=1}^T\left(Q^*\left(s_t, a_t\right)-\gamma V^*\left(s_t\right)-r\left(s_t, a_t\right)\right) \\
& \leq 2 \operatorname{sp}\left(v^*\right) \sqrt{2 T \ln \frac{1}{\delta}}+2 \operatorname{sp}\left(v^*\right) .
\end{aligned}
$$

これはベルマン方程式とAzuma-Hoeffdingでいけます．

**証明**

ベルマン方程式より，
$$
Q^*\left(s_t, a_t\right)-\gamma V^*\left(s_t\right)-r\left(s_t, a_t\right)=
\gamma\left(\mathbb{E}_{s^{\prime} \sim p\left(\cdot \mid s_t, a_t\right)}\left[V^*\left(s^{\prime}\right)\right]-V^*\left(s_t\right)\right)
$$
です．よって，
$$
\sum_{t=1}^T\left(Q^*\left(s_t, a_t\right)-\gamma V^*\left(s_t\right)-r\left(s_t, a_t\right)\right)=\gamma \sum_{t=1}^T\left(\mathbb{E}_{s^{\prime} \sim p\left(\cdot \mid s_t, a_t\right)}\left[V^*\left(s^{\prime}\right)\right]-V^*\left(s_{t+1}\right)\right)+\gamma \sum_{t=1}^T\left(V^*\left(s_{t+1}\right)-V^*\left(s_t\right)\right)
$$

が成り立ちます．後はHoeffdingとSpanでバウンドすれば終わりです．

---

後は全ての項を合体すれば，リグレットバウンドが出てきます．

このアルゴリズムについて，次のリグレットが成立します：

---

**リグレットのバウンド**

Weakly communicatingなMDPについて，$H=\min \left\{\sqrt{\frac{\operatorname{sp}\left(v^*\right) T}{S A}},\left(\frac{T}{S A \ln \frac{4 T}{\delta}}\right)^{\frac{1}{3}}\right\}$のとき，上のアルゴリズムは次のリグレットを達成します：

$$
\mathcal{O}\left(\sqrt{\operatorname{sp}\left(v^*\right) S A T}+\operatorname{sp}\left(v^*\right)\left(T^{\frac{2}{3}}\left(S A \ln \frac{T}{\delta}\right)^{\frac{1}{3}}+\sqrt{T \ln \frac{1}{\delta}}\right)\right)
$$

**注意：**

* $H$でスパンの情報を使う必要があります．
* リグレットはスパンに依存しており，問題依存のバウンドになってます．ちょっと偉い．

---

## 平均報酬に拡張

$$
\begin{aligned}
R_T= & \sum_{t=1}^T\left(J^*-r\left(s_t, a_t\right)\right) \\
= & \sum_{t=1}^T\left(J^*-(1-\gamma) V^*\left(s_t\right)\right) \\
& +\sum_{t=1}^T\left(V^*\left(s_t\right)-Q^*\left(s_t, a_t\right)\right) \\
& +\sum_{t=1}^T\left(Q^*\left(s_t, a_t\right)-\gamma V^*\left(s_t\right)-r\left(s_t, a_t\right)\right) .
\end{aligned}
$$

* 最後に$\gamma = 1 - \frac{1}{H}$を代入し，適切な$H$を選択することで証明が完了します．
* 最初の項が$\frac{T}{H}$，次の項が$\sqrt{HT}$でバウンドされます

このそれぞれの項をバウンドしていきましょう．

---

### １つ目のバウンド

まず，次が成立します （[RL_useful_lemma/RL_AverageReward_lemma.ipynb](RL_useful_lemma/RL_AverageReward_lemma.ipynb)を見てね）:

1. $\left|J^*-(1-\gamma) V^*(s)\right| \leq(1-\gamma) \operatorname{sp}\left(v^*\right), \forall s \in \mathcal{S}$,
2. $\operatorname{sp}\left(V^*\right) \leq 2 \operatorname{sp}\left(v^*\right)$.

明らかにこれを使えばリグレットの１つ目はバウンドできます．
